# Cómo los sensores "ven" el movimiento humano — Actividades prácticas (IMU)
**Duración total sugerida:** 1h30 (4 actividades cortas)

Este cuaderno contiene cuatro actividades pensadas para estudiantes que **no necesitan** programar desde cero.
Cada bloque tiene **parámetros editables** y **comentarios** para explorar rápidamente.

**Formato general de los datos (flexible):**
- Columna 0: `Timestamp` (segundos o milisegundos).
- Columnas típicas: `AccX, AccY, AccZ`, `GyrX, GyrY, GyrZ`, `MagX, MagY, MagZ`.
- Opcionalmente: `QuatW, QuatX, QuatY, QuatZ` (por sensor).
- Si usas **dos sensores** (muslo y tibia), usualmente habrá sufijos/prefijos por sensor, p. ej. `Thigh_AccX`, `Shank_AccX`, `S1_AccX`, `S2_AccX`, etc.

> **Tip:** Si tu CSV tiene columnas extra (PacketCounter, Status…), el código ignora lo que no necesita.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, butter, filtfilt

# Configuración de gráficos (sin estilos ni colores específicos)
plt.rcParams['figure.figsize'] = (9, 4)

## 0) Carga de datos y exploración rápida
1. Cambia la ruta del archivo a tu CSV.
2. Revisa qué columnas hay y sus primeros registros.

In [ ]:
# === EDITA AQUÍ ===
CSV_PATH = '/mnt/data/tu_archivo.csv'  # Cambia a la ruta de tu CSV

# === CARGA ===
df = pd.read_csv(CSV_PATH)

# Aceptamos Timestamp en la columna 0 con cualquier nombre, lo renombramos internamente
if df.columns[0].lower() != 'timestamp':
    df = df.rename(columns={df.columns[0]: 'Timestamp'})

# Convertir a segundos si detectamos milisegundos (heurística simple)
if df['Timestamp'].max() > 1e5:  # probablemente está en milisegundos
    df['Timestamp'] = df['Timestamp'] / 1000.0

print("Columnas detectadas:")
print(list(df.columns))

print("\nPrimeras filas:")
display(df.head(5))

In [ ]:
def butter_lowpass_filter(x, fs, cutoff=5.0, order=2):
    nyq = 0.5 * fs
    b, a = butter(order, cutoff/nyq, btype='low', analog=False)
    return filtfilt(b, a, x)

def estimate_sampling_rate(t):
    # t: vector de tiempo en segundos
    dt = np.diff(t)
    dt = dt[dt > 0]
    return 1.0 / np.median(dt) if len(dt) else np.nan

## 1) Visualiza cualquier señal vs. tiempo (Acc / Gyr / Mag / Euler / lo que tengas)
- Elige el **nombre exacto** de una columna (p. ej., `AccZ`, `Thigh_AccZ`, `GyrY`, etc.).
- Se graficará contra `Timestamp`.

In [ ]:
# === EDITA AQUÍ (elige una columna para ver) ===
COLUMNA_A_PLOT = 'AccZ'  # cambia por la columna que quieras visualizar

assert 'Timestamp' in df.columns, "No se encontró la columna de tiempo 'Timestamp'"
assert COLUMNA_A_PLOT in df.columns, f"No se encontró la columna: {COLUMNA_A_PLOT}"

t = df['Timestamp'].to_numpy()
y = df[COLUMNA_A_PLOT].to_numpy()

plt.figure()
plt.plot(t, y)
plt.xlabel('Tiempo [s]')
plt.ylabel(COLUMNA_A_PLOT)
plt.title(f'Señal {COLUMNA_A_PLOT} vs. tiempo')
plt.show()

## 2) ¿Cuántos **pasos** y **giros**? (fácil y directo)
**Pasos (STEP COUNT):**
- Método simple por **detección de picos** en una señal de aceleración (idealmente vertical).
- Opcionalmente filtra con pasa-bajas (5 Hz) para limpiar ruido.
- Si el sensor está en **una pierna**: cada pico ≈ 1 paso.
- Si el sensor está en **tronco/cadera**: cada pico fuerte ≈ 2 pasos → puedes dividir por 2.

**Giros (TURN COUNT):**
- Integramos una **componente del giroscopio** (elegir eje) para obtener ángulo acumulado.
- Contamos cuántas **vueltas completas** (|ángulo acumulado| / 360).

In [ ]:
# === EDITA AQUÍ ===
ACC_EJE = 'AccZ'         # eje para pasos (elige AccX/AccY/AccZ o magnitud)
GYR_EJE = 'GyrZ'         # eje de giros (usualmente Z ~ yaw si sensor está vertical)
SENSOR_EN_TRONCO = False # si True, dividir pasos por 2 (porque la oscilación incluye ambos pies)

# Si no sabes la orientación, puedes usar la magnitud de aceleración:
# MAG_ACC = True usa sqrt(AccX^2 + AccY^2 + AccZ^2)
MAG_ACC = False

# Parámetros de picos
MIN_DIST_S = 0.4   # separación mínima entre pasos [s] (ajusta según cadencia)
THRESH = None      # si None, se calcula automáticamente

# === PREPARA SEÑAL ===
t = df['Timestamp'].to_numpy()
fs = estimate_sampling_rate(t)
assert np.isfinite(fs) and fs > 0, "No se pudo estimar el muestreo"

if MAG_ACC:
    # requiere AccX, AccY, AccZ
    for c in ['AccX','AccY','AccZ']:
        assert c in df.columns, f"Falta columna {c} para magnitud"
    acc = np.sqrt(df['AccX']**2 + df['AccY']**2 + df['AccZ']**2).to_numpy()
    acc_name = '||Acc||'
else:
    assert ACC_EJE in df.columns, f"No se encontró columna {ACC_EJE}"
    acc = df[ACC_EJE].to_numpy()
    acc_name = ACC_EJE

# Filtra suavemente
acc_f = butter_lowpass_filter(acc, fs, cutoff=5.0, order=2)

# Detección de picos
min_distance_samples = int(MIN_DIST_S * fs)
if THRESH is None:
    # umbral automático: media + 0.5*std (ajústalo si hace falta)
    THRESH = np.mean(acc_f) + 0.5*np.std(acc_f)

peaks, props = find_peaks(acc_f, height=THRESH, distance=max(1, min_distance_samples))
n_steps = len(peaks)
if SENSOR_EN_TRONCO:
    n_steps = int(round(n_steps / 2))

print(f"Frecuencia de muestreo estimada: {fs:.2f} Hz")
print(f"Pasos detectados ({acc_name}): {n_steps}")

# Grafica con los picos
plt.figure()
plt.plot(t, acc_f, label=f'{acc_name} filtrada')
plt.plot(t[peaks], acc_f[peaks], 'o', label='picos')
plt.xlabel('Tiempo [s]')
plt.ylabel(acc_name)
plt.title('Detección de pasos')
plt.legend()
plt.show()

# === GIROS: integra giroscopio elegido ===
assert GYR_EJE in df.columns, f"No se encontró columna {GYR_EJE}"
gyr = df[GYR_EJE].to_numpy()  # [deg/s] o [rad/s], depende del CSV
# Intento de detección de unidades: si valores son muy pequeños, asumimos rad/s y convertimos
if np.nanmax(np.abs(gyr)) < 20:  # heurística simple
    gyr_deg_s = np.rad2deg(gyr)
else:
    gyr_deg_s = gyr

# Integración simple por trapecios para ángulo acumulado
theta_deg = np.zeros_like(gyr_deg_s)
theta_deg[1:] = np.cumsum(0.5*(gyr_deg_s[1:]+gyr_deg_s[:-1]) * np.diff(t))

# Cuenta vueltas completas (360°)
turns = int(np.floor(np.abs(theta_deg[-1]) / 360.0))
print(f"Giros estimados (vueltas completas): {turns}")

plt.figure()
plt.plot(t, theta_deg)
plt.xlabel('Tiempo [s]')
plt.ylabel('Ángulo acumulado [deg]')
plt.title(f'Integración de {GYR_EJE} (estimación de giros)')
plt.show()

## 3) Ángulo de **rodilla** (flexión/extensión) — enfoque rápido y visual
Hay dos rutas:

**A) Con cuaterniones (recomendado si los tienes):**  
- Usa columnas de cuaterniones para **muslo** y **tibia** (por ejemplo: `Thigh_QuatW/X/Y/Z` y `Shank_QuatW/X/Y/Z`).  
- Calcula la **orientación relativa**: `q_rel = q_shank * conj(q_thigh)` y extrae un ángulo tipo *pitch*.

**B) Exploratorio por ejes (para pensar y probar):**  
- Permite seleccionar **ejes simples** (por ejemplo `EulerPitch` de cada sensor o un eje del giroscopio integrado) y ver cómo se comporta la señal resultante.
- **Idea pedagógica:** probar combinaciones y ver por qué **colocar los sensores de lado, a mitad del muslo y tibia** produce la señal más limpia de flexo-extensión.

In [ ]:
# === EDITA AQUÍ: NOMBRES DE COLUMNAS ===
# Opción A: columnas de cuaterniones por sensor (si existen)
THIGH_QUAT_COLS = ['Thigh_QuatW','Thigh_QuatX','Thigh_QuatY','Thigh_QuatZ']  # cambia según tu CSV
SHANK_QUAT_COLS = ['Shank_QuatW','Shank_QuatX','Shank_QuatY','Shank_QuatZ']  # cambia según tu CSV

# Opción B: columnas "exploratorias" por ejes (si no hay cuaterniones o quieres experimentar)
# Por ejemplo, podrías tener Euler o elegir un eje de giroscopio integrado
THIGH_AXIS_COL = 'Thigh_EulerPitch'   # o 'Thigh_GyrY', etc.
SHANK_AXIS_COL = 'Shank_EulerPitch'
USE_OPTION = 'A'  # 'A' para cuaterniones, 'B' para ejes

In [ ]:
def quat_conj(q):
    # q: (..., 4) [w, x, y, z]
    qc = q.copy()
    qc[..., 1:] *= -1
    return qc

def quat_mul(q1, q2):
    # Hamilton product
    w1,x1,y1,z1 = q1[...,0], q1[...,1], q1[...,2], q1[...,3]
    w2,x2,y2,z2 = q2[...,0], q2[...,1], q2[...,2], q2[...,3]
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2
    return np.stack([w,x,y,z], axis=-1)

def quat_to_euler_xyz(q):
    # Convierte a Euler (XYZ) en radianes; devuelve roll (x), pitch (y), yaw (z)
    w, x, y, z = q[...,0], q[...,1], q[...,2], q[...,3]
    # roll (x)
    sinr_cosp = 2*(w*x + y*z)
    cosr_cosp = 1 - 2*(x*x + y*y)
    roll = np.arctan2(sinr_cosp, cosr_cosp)
    # pitch (y)
    sinp = 2*(w*y - z*x)
    sinp = np.clip(sinp, -1, 1)
    pitch = np.arcsin(sinp)
    # yaw (z)
    siny_cosp = 2*(w*z + x*y)
    cosy_cosp = 1 - 2*(y*y + z*z)
    yaw = np.arctan2(siny_cosp, cosy_cosp)
    return np.stack([roll, pitch, yaw], axis=-1)

def unwrap_deg(a_deg):
    a = np.deg2rad(a_deg)
    a = np.unwrap(a)
    return np.rad2deg(a)

t = df['Timestamp'].to_numpy()

if USE_OPTION.upper() == 'A':
    # --- Usando cuaterniones ---
    for cols in (THIGH_QUAT_COLS, SHANK_QUAT_COLS):
        for c in cols:
            assert c in df.columns, f"Falta columna: {c}"
    q_thigh = df[THIGH_QUAT_COLS].to_numpy()
    q_shank = df[SHANK_QUAT_COLS].to_numpy()

    # Normaliza por si acaso
    q_thigh = q_thigh / np.linalg.norm(q_thigh, axis=1, keepdims=True)
    q_shank = q_shank / np.linalg.norm(q_shank, axis=1, keepdims=True)

    # Orientación relativa (de muslo a tibia)
    q_rel = quat_mul(q_shank, quat_conj(q_thigh))

    # Euler relativo (XYZ); elegimos **pitch (eje Y)** como flexo-extensión aproximada
    euler_rel = quat_to_euler_xyz(q_rel)   # rad
    knee_pitch_deg = np.rad2deg(euler_rel[:,1])
    knee_pitch_deg = unwrap_deg(knee_pitch_deg)

    plt.figure()
    plt.plot(t, knee_pitch_deg)
    plt.xlabel('Tiempo [s]'); plt.ylabel('Rodilla [deg] (aprox. pitch relativo)')
    plt.title('Ángulo de rodilla (flexo-extensión) — opción A (cuaterniones)')
    plt.show()

else:
    # --- Exploratorio por ejes ---
    assert THIGH_AXIS_COL in df.columns and SHANK_AXIS_COL in df.columns, "Revisa columnas de ejes"
    thigh_axis = df[THIGH_AXIS_COL].to_numpy()
    shank_axis = df[SHANK_AXIS_COL].to_numpy()

    # Si pareciera estar en radianes (magnitudes pequeñas), convertir a grados
    if np.nanmax(np.abs(thigh_axis)) < 6:  # heurística
        thigh_axis = np.rad2deg(thigh_axis)
    if np.nanmax(np.abs(shank_axis)) < 6:
        shank_axis = np.rad2deg(shank_axis)

    knee_deg = unwrap_deg(shank_axis - thigh_axis)

    plt.figure()
    plt.plot(t, knee_deg)
    plt.xlabel('Tiempo [s]'); plt.ylabel('Rodilla [deg] (ejes seleccionados)')
    plt.title('Ángulo de rodilla — opción B (explorar ejes)')
    plt.show()

print("💡 Sugerencia pedagógica: coloca los sensores de **lado**, a mitad del muslo y la tibia. Prueba distintos ejes/órdenes y observa cuál produce una curva de flexo-extensión más limpia.")

## 4) Explora y cuestiona (preguntas guía para el grupo)
- ¿Qué pasa si cambio el eje de aceleración para contar pasos? ¿Cuál funciona mejor y por qué?
- ¿Usar magnitud de aceleración ayuda cuando no conozco la orientación del sensor?
- ¿Qué eje de giroscopio representa mejor los **giros** (yaw)? ¿Qué pasa si el sensor no está vertical?
- En rodilla, ¿qué combinaciones de ejes dan una curva “parecida” a flexo-extensión? ¿Por qué la colocación lateral ayuda?
- ¿Qué filtros o parámetros (umbral, distancia mínima entre picos) mejoran la detección de pasos sin perder eventos?